In [1]:
df = pd.read_csv('train.csv')
conversion_dict = {0: 'Real', 1: 'Fake'}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

NameError: name 'pd' is not defined

In [8]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75)

In [9]:
vec_train=tfidf_vectorizer.fit_transform(x_train.values.astype("U"))
vec_test=tfidf_vectorizer.transform(x_test.values.astype("U"))

In [10]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(vec_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [11]:
y_pred = pac.predict(vec_test)
score = accuracy_score(y_test, y_pred)
print(f"PAC Accuracy: {round(score*100,2)}%")

PAC Accuracy: 96.23%


In [12]:
X = tfidf_vectorizer.transform(df['text'].values.astype('U'))

In [13]:
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f"K Fold Accuracy: {round(scores.mean()*100,2)}%")

K Fold Accuracy: 96.2%


In [14]:
df_true = pd.read_csv('True.csv')
df_true['label'] = 'Real'
df_true_rep = [df_true['text'][i].replace('WASHINGTON (Reuters) - ', '').replace('LONDON (Reuters) - ', '').replace('(Reuters) - ', '') for i in range(len(df_true['text']))]
df_true['text'] = df_true_rep
df_fake = pd.read_csv('Fake.csv')
df_fake['label'] = 'Fake'
df_final = pd.concat([df_true, df_fake])
df_final = df_final.drop(['subject', 'date'], axis=1)
df_fake

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


In [15]:
def findlabel(newtext):
    vec_newtest = tfidf_vectorizer.transform([newtext])
    y_pred1 = pac.predict(vec_newtest)
    return y_pred1[0]

In [16]:
findlabel((df_true['text'][0]))

'Real'

In [17]:
findlabel((df_fake['text'][0]))

'Fake'

In [18]:
# Iterate on all the true articles, and verify how much percentage are those predicted to be true
sum([1 if findlabel((df_true['text'][i])) == 'Real' else 0 for i in range(len(df_true['text']))]) / df_true['text'].size

0.7244245225755241

In [19]:
sum([1 if findlabel((df_fake['text'][i])) == 'Fake' else 0 for i in range(len(df_fake['text']))]) / df_fake['text'].size

0.6946467356586176

In [22]:
# Save the model in disk
import joblib
filename = 'model.sav'
joblib.dump(pac, filename)

# Export vectorizer
filename_vectorizer = 'vectorizer.sav'
joblib.dump(tfidf_vectorizer, filename_vectorizer)

['vectorizer.sav']

In [21]:
findlabel((df_fake['text'][0]))

'Fake'